# Complex solution implementation documentation 

In [ ]:
import sympy as sym
sym.init_printing()

## Gibbs free energy with ordering parameters

The extensive (mass dependent) Gibbs free energy will be denoted:  
$G\left( {T,P,{\bf{n}},{\bf{s}}} \right)$,  
where $\bf{n}$ is a vector of endmember components of length *nc* and $\bf{s}$ is a vector of dependent (ordering) parameters of length *ns*.  

As a concrete example, *nc* is set to 5 and *ns* is set to 3

In [ ]:
nc = 2
ns = 1

Construct a vector of endmember component mole numbers ...

In [ ]:
T,P = sym.symbols('T P')
c_str = ''
for i in range(1,nc+1):
    c_str += 'n' + str(i) + ' '
n = sym.Matrix(sym.symbols(c_str))

... and an argument list for the implicit ordering functions

In [ ]:
s_args = (T,P)
s_args += tuple([n[i] for i in range(0,nc)])

Construct the ordering functions, $Os_i$, and store them in a matrix, $Os$, and construct ordering variables, $s_i$, for the Gibbs function and store these in a matrix, $s$.  

This is done so that we can take derivatives of the Gibbs function with respect to ordering variables and to use ordering functions to evaluate exterior derivatives over the homogeneous equilibrium ordering surface manifold.

In [ ]:
Os_list = []
s_str = ''
for i in range(1,ns+1):
    Os_list.append(sym.Function('Os'+str(i))(*s_args))
    s_str += 's' + str(i) + ' '
Os = sym.Matrix(Os_list)
s = sym.Matrix(sym.symbols(s_str if ns > 1 else [s_str]))

Construct the argument list for the Gibbs free energy function

In [ ]:
g_args = s_args + tuple([s[i] for i in range(0,ns)])
g_args

Construct the Gibbs free energy function

In [ ]:
G = sym.Function('G')(*g_args)
G

## The total derivative of the Gibbs free energy is  

$dG = \frac{{\partial G}}{{\partial T}}dT + \frac{{\partial G}}{{\partial P}}dP + \frac{{\partial G}}{{\partial {\bf{n}}}}d{\bf{n}} + \frac{{\partial G}}{{\partial {\bf{s}}}}d{\bf{s}}$  

## At homogeneous equilibrium, $\frac{{\partial G}}{{\partial {\bf{s}}}} = {\bf{0}}$:
(this expression is non-linear and must be solved iteratively, e.g., Newton's method)

In [ ]:
hEQ = sym.Matrix.zeros(rows=ns,cols=1)
sym.Eq(G.diff(s),hEQ)

## Consequently, from the total derivative of the Gibbs free energy
along the homogenious equilibrium manifold ...

### (1-1) ${\left. {\frac{{\partial G}}{{\partial T}}} \right|_{eq}} = \frac{{\partial G}}{{\partial T}}$  

### (1-2) ${\left. {\frac{{\partial G}}{{\partial P}}} \right|_{eq}} = \frac{{\partial G}}{{\partial P}}$  

### (1-3) ${\left. {\frac{{\partial G}}{{\partial \bf{n}}}} \right|_{eq}} = \frac{{\partial G}}{{\partial \bf{n}}}$ 

## First derivatives of the ordering parameters  

$d\frac{{\partial G}}{{\partial {\bf{s}}}} = {\bf{0}} = \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}dT + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}dP + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}d{\bf{n}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}d{\bf{s}}$

Compute $\frac{{d{\bf{s}}}}{{dT}}$  

### ${\bf{0}} = \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}$
Code generates a vector of length *ns*. Note that the vector orientation (horizontal) follows the convention (p. 52) in Graham, A. (1981) *Kronecker Products and Matrix Calculus with Applications*, John Wiley and Sons, New York, 130pp.

In [ ]:
dsdt = Os.diff(T)
d2gdtds = sym.Matrix([G.diff(T).diff(x) for x in s])
d2gds2 = sym.Matrix([[G.diff(x).diff(y) for x in s] for y in s])
dsdt = sym.Matrix(list(sym.solve(sym.Eq(d2gdtds+d2gds2*dsdt,hEQ), dsdt).values())).T
dsdt

Compute $\frac{{d{\bf{s}}}}{{dP}}$  

### ${\bf{0}} = \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}$  
Code generates a vector of length *ns*. 

In [ ]:
dsdp = Os.diff(P)
d2gdpds = sym.Matrix([G.diff(P).diff(x) for x in s])
dsdp = sym.Matrix(list(sym.solve(sym.Eq(d2gdpds+d2gds2*dsdp,hEQ), dsdp).values())).T
dsdp

Compute $\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$  

### ${\bf{0}} = \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$   
Code generates a matrix with *nc* rows and *ns* columns, as per the convention (p. 52) discussed in Graham (1981, *loc. cit.*). 

In [ ]:
result = []
for i in range(0,nc):
    dsdn = Os.diff(n[i])
    d2gdnds = sym.Matrix([G.diff(n[i]).diff(x) for x in s])
    result.append(list(sym.solve(sym.Eq(d2gdnds+d2gds2*dsdn,hEQ), dsdn).values()))
dsdn = sym.Matrix(result)
dsdn if ns == 1 else dsdn.shape

## The second total derivative of the Gibbs free energy 

Assumptions:
- Only the upper triangle of derivatives are computed, i.e. derivative are always taken in the order $T$, $P$, $\bf{n}$, e.g., $TT$, $TP$, $T\bf{n}$, $PP$, $P\bf{n}$, $\bf{nn}$
- Variables are ordered as (1), (2), i.e., ${T_1}{T_2}$

${d^2}G = \frac{{\partial G}}{{\partial {T_1}\partial {T_2}}}d{T_1}d{T_2} + \frac{{\partial G}}{{\partial {T_1}\partial {P_2}}}d{T_1}d{P_2} + \frac{{\partial G}}{{\partial {T_1}\partial {{\bf{n}}_2}}}d{T_1}d{{\bf{n}}_2} + \frac{{\partial G}}{{\partial {T_1}\partial {{\bf{s}}_2}}}d{T_1}d{{\bf{s}}_2} + \frac{{\partial G}}{{\partial {T_1}}}{d^2}{T_1} + \frac{{\partial G}}{{\partial {T_2}\partial {P_1}}}d{P_1}d{T_2} + \frac{{\partial G}}{{\partial {P_1}\partial {P_2}}}d{P_1}d{P_2} + \frac{{\partial G}}{{\partial {P_1}\partial {{\bf{n}}_2}}}d{P_1}d{{\bf{n}}_2} + \frac{{\partial G}}{{\partial {P_1}\partial {{\bf{s}}_2}}}d{P_1}d{{\bf{s}}_2} + \frac{{\partial G}}{{\partial {P_1}}}{d^2}{P_1} + \frac{{\partial G}}{{\partial {T_2}\partial {{\bf{n}}_1}}}d{T_2}d{{\bf{n}}_1} + \frac{{\partial G}}{{\partial {P_2}\partial {{\bf{n}}_1}}}d{P_2}d{{\bf{n}}_1} + \frac{{\partial G}}{{\partial {{\bf{n}}_1}\partial {{\bf{n}}_2}}}d{{\bf{n}}_1}d{{\bf{n}}_2} + \frac{{\partial G}}{{\partial {{\bf{n}}_1}\partial {{\bf{s}}_2}}}d{{\bf{n}}_1}d{{\bf{s}}_2} + \frac{{\partial G}}{{\partial {{\bf{n}}_1}}}{d^2}{{\bf{n}}_1} + \frac{{\partial G}}{{\partial {T_2}\partial {{\bf{s}}_1}}}d{T_2}d{{\bf{s}}_1} + \frac{{\partial G}}{{\partial {P_2}\partial {{\bf{s}}_1}}}d{P_2}d{{\bf{s}}_1} + \frac{{\partial G}}{{\partial {{\bf{n}}_2}\partial {{\bf{s}}_1}}}d{{\bf{n}}_2}d{{\bf{s}}_1} + \frac{{\partial G}}{{\partial {{\bf{s}}_1}\partial {{\bf{s}}_2}}}d{{\bf{s}}_1}d{{\bf{s}}_2} + \frac{{\partial G}}{{\partial {{\bf{s}}_1}}}{d^2}{{\bf{s}}_1}$  

## Consequently,
along the homogenious equilibrium manifold (*eq*) - $\frac{{\partial G}}{{\partial {{\bf{s}}}}}$ is assumed to be zero: 

### (2-1) ${\left. {\frac{{{\partial ^2}G}}{{\partial {T^2}}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial {T^2}}} + 2\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}}$  
Code generates a scalar.

In [ ]:
result  = G.diff(T,2)
result += 2*(dsdt*d2gdtds)[0]
result += (dsdt*d2gds2*dsdt.T)[0]
result.simplify()

### (2-2) ${\left. {\frac{{{\partial ^2}G}}{{\partial T\partial P}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial T\partial P}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}}$  
Code generates a scalar.

In [ ]:
result  = G.diff(T,P)
result += (dsdp*d2gdtds)[0]
result += (dsdt*d2gdpds)[0]
result += (dsdt*d2gds2*dsdp.T)[0]
result.simplify()

### (2-3) ${\left. {\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{n}}}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$  
Code generates a vector of length *nc*, oriented vertically using the convention discussed in Graham (1981, *loc. cit.*, p. 52)

In [ ]:
d2gdnds = sym.Matrix([[G.diff(y).diff(x) for x in n] for y in s])
result  = G.diff(T,n)
result += dsdn*d2gdtds
result += (dsdt*d2gdnds).T
result += (dsdt*d2gds2*(dsdn.T)).T
result.simplify()

### (2-4) ${\left. {\frac{{{\partial ^2}G}}{{\partial {P^2}}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial {P^2}}} + 2\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}}$  
Code generates a scalar.

In [ ]:
result  = G.diff(P,2)
result += 2*(dsdp*d2gdpds)[0]
result += (dsdp*d2gds2*dsdp.T)[0]
result.simplify()

### (2-5) ${\left. {\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{n}}}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$  
Code generates a vector of length *nc*.

In [ ]:
result  = G.diff(P,n)
result += dsdn*d2gdpds
result += (dsdp*d2gdnds).T
result += (dsdp*d2gds2*(dsdn.T)).T
result.simplify()

### (2-6) ${\left. {\frac{{{\partial ^2}G}}{{\partial {{\bf{n}}^2}}}} \right|_{eq}} = \frac{{{\partial ^2}G}}{{\partial {{\bf{n}}^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$  
Code generates a symmetric matrix of size *nc*.

In [ ]:
result  = sym.Matrix([[G.diff(x,y) for x in n] for y in n])
result += dsdn*d2gds2*(dsdn.T)
result += 2*dsdn*d2gdnds
result if nc < 3 else result.shape

## Second derivatives of the ordering parameters   

${d^2}\frac{{\partial G}}{{\partial s}} = \frac{{{\partial ^3}G}}{{\partial {T^2}\partial s}}dTdT + 2\frac{{{\partial ^3}G}}{{\partial T\partial P\partial s}}dTdP + 2\frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial s}}dTd{\bf{n}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}dTd{\bf{s}} + \frac{{{\partial ^2}G}}{{\partial T\partial s}}{d^2}T + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial s}}dPdP + 2\frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial s}}dPd{\bf{n}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}dPd{\bf{s}} + \frac{{{\partial ^2}G}}{{\partial P\partial s}}{d^2}P + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial s}}d{\bf{n}}d{\bf{n}} + 2\frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}d{\bf{n}}d{\bf{s}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial s}}{d^2}{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}d{\bf{s}}d{\bf{s}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}{d^2}{\bf{s}}$

## $\frac{{{d^2}{\bf{s}}}}{{d{T^2}}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial {T^2}\partial s}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}}$  
Code generates a vcetor of length *ns*, oriented horizontally.

In [ ]:
d2sdt2 = Os.diff(T,2)
d3gdtds2 = sym.Matrix([[G.diff(T).diff(x).diff(y) for x in s] for y in s])
d3gds3 = []
for x in s:
    d3gds3.append(sym.Matrix([[G.diff(x).diff(y).diff(z) for y in s] for z in s]))
term = []
for i in range (0,ns):
    term.append(dsdt*d3gds3[i]*dsdt.T)
term  = sym.Matrix(term)
term += sym.Matrix([G.diff(T,2).diff(x) for x in s])
term += 2*d3gdtds2*(dsdt.T)
term += d2gds2*d2sdt2
d2sdt2 = sym.Matrix(list(sym.solve(sym.Eq(term,hEQ), d2sdt2).values())).T
d2sdt2 if ns == 1 else d2sdt2.shape

## $\frac{{d{{\bf{s}}^2}}}{{dTdP}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial T\partial P\partial s}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}}$  
Code generates a vector of length *ns*, oriented horizontally.

In [ ]:
d2sdtdp = Os.diff(T).diff(P)
d3gdpds2 = sym.Matrix([[G.diff(P).diff(x).diff(y) for x in s] for y in s])
term = []
for i in range (0,ns):
    term.append(dsdp*d3gds3[i]*dsdt.T)
term  = sym.Matrix(term)
term += sym.Matrix([G.diff(T).diff(P).diff(x) for x in s])
term += d3gdpds2*(dsdt.T)
term += d3gdtds2*(dsdp.T)
term += d2gds2*d2sdtdp
d2sdtdp = sym.Matrix(list(sym.solve(sym.Eq(term,hEQ), d2sdtdp).values())).T
d2sdtdp if ns == 1 else d2sdtdp.shape

## $\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial s}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}}$  
Generates a matrix with *nc* rows and *ns* columns.

In [ ]:
d2sdtdn = sym.Matrix([[Os[i].diff(T).diff(x) for i in range(0,ns)] for x in n])
d3gdnds2 = []
for i in range(0,nc):
    d3gdnds2.append(sym.Matrix([[G.diff(n[i]).diff(x).diff(y) for x in s] for y in s]))
d3gdtdnds = sym.Matrix([[G.diff(T).diff(x).diff(y) for x in n] for y in s])

rows = []
for i in range(0,nc):
    term = []
    for j in range (0,ns):
        term.append(dsdn.row(i)*d3gds3[j]*(dsdt.T))
    term  = sym.Matrix(term)
    term += d3gdnds2[i]*(dsdt.T)
    term += d3gdtds2*dsdn.row(i).T
    term += d3gdtdnds.col(i)
    term += d2gds2*(d2sdtdn.row(i).T)
    rows.append(list(sym.solve(sym.Eq(term,hEQ), d2sdtdn.row(i)).values()))
d2sdtdn = sym.Matrix(rows)
d2sdtdn if ns == 1 else d2sdtdn.shape

## $\frac{{{d^2}{\bf{s}}}}{{d{P^2}}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial {P^2}\partial s}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}}$

In [ ]:
d2sdp2 = Os.diff(P,2)
term = []
for i in range (0,ns):
    term.append(dsdp*d3gds3[i]*dsdp.T)
term  = sym.Matrix(term)
term += sym.Matrix([G.diff(P,2).diff(x) for x in s])
term += 2*d3gdpds2*(dsdp.T)
term += d2gds2*d2sdp2
d2sdp2 = sym.Matrix(list(sym.solve(sym.Eq(term,hEQ), d2sdp2).values())).T
d2sdp2 if ns == 1 else d2sdp2.shape

## $\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial s}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}}$

In [ ]:
d2sdpdn = sym.Matrix([[Os[i].diff(P).diff(x) for i in range(0,ns)] for x in n])
d3gdpdnds = sym.Matrix([[G.diff(P).diff(x).diff(y) for x in n] for y in s])

rows = []
for i in range(0,nc):
    term = []
    for j in range (0,ns):
        term.append(dsdn.row(i)*d3gds3[j]*(dsdp.T))
    term  = sym.Matrix(term)
    term += d3gdnds2[i]*(dsdp.T)
    term += d3gdpds2*dsdn.row(i).T
    term += d3gdpdnds.col(i)
    term += d2gds2*(d2sdpdn.row(i).T)
    rows.append(list(sym.solve(sym.Eq(term,hEQ), d2sdpdn.row(i)).values()))
d2sdpdn = sym.Matrix(rows)
d2sdpdn if ns == 1 else d2sdpdn.shape

## $\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}}$  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial s}} + 2\frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}}$  
Code generates a list of matrices. The matrices are *nc* by *nc*, the list of of length *ns*.

In [ ]:
rows = []
for i in range(0,nc):
    cols = []
    for j in range(0,nc):
        term = []
        for k in range (0,ns):
            term.append((dsdn.row(i))*d3gds3[k]*(dsdn.row(j).T))
        term  = sym.Matrix(term)
        term += G.diff(n[i]).diff(n[j]).diff(s)
        term += 2*d3gdnds2[i]*(dsdn.row(j).T)
        term += d2gds2*Os.diff(n[i]).diff(n[j])
        cols.append(sym.Matrix(list(sym.solve(sym.Eq(term,hEQ), Os.diff(n[i]).diff(n[j])).values())))
    rows.append(cols)
d2sdn2 = []
for i in range(0,ns):
    d2sdn2.append(sym.Matrix([[rows[j][k][i] for j in range(0,nc)] for k in range(0,nc)]))
d2sdn2 if ns == 1 else len(d2sdn2)

## The third total derivative of the Gibbs free energy is:  

On the assumption that the derivatives are always ordered as $T$, $P$, $\bf{n}$, i.e., we compute *only* the upper triangular wedge of the cube $d^3G$ (derivatives with respect to $\bf{s}$ are not computed).  The order of differentiation is always 1,2,3, and is implied in the following expression unless ambiguous, in which case derivatives are labeled:  

${d^3}G = \frac{{{\partial ^3}G}}{{\partial {T^3}}}dTdTdT + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial P}}dTdTdP + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{n}}}}dTdTd{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}dTdTd{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}dTd{\bf{s}}dT + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}d{\bf{s}}dTdT + \frac{{{\partial ^3}G}}{{\partial T\partial {P^2}}}dTdPdP + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{n}}}}dTdPd{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}dTdPd{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}dTd{\bf{s}}dP + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}d{\bf{s}}dTdP + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{n}}^2}}}dTd{\bf{n}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}dTd{\bf{n}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}dTd{\bf{s}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}d{\bf{s}}dTd{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}dTd{\bf{s}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}d{\bf{s}}dTd{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}d{\bf{s}}d{\bf{s}}dT + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}d{T_1}{d^2}{{\bf{s}}_{2,3}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}d{T_2}{d^2}{{\bf{s}}_{1,3}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}d{T_3}{d^2}{{\bf{s}}_{1,2}} + \frac{{{\partial ^3}G}}{{\partial {P^3}}}dPdPdP + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{n}}}}dPdPd{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}dPdPd{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}dPd{\bf{s}}dP + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}d{\bf{s}}dPdP + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{n}}^2}}}dPd{\bf{n}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}dPd{\bf{n}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}dPd{\bf{s}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}d{\bf{s}}dPd{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}dPd{\bf{s}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}d{\bf{s}}dPd{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}d{\bf{s}}d{\bf{s}}dP + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}d{P_1}{d^2}{{\bf{s}}_{2,3}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}d{P_2}{d^2}{{\bf{s}}_{1,3}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}d{P_3}{d^2}{{\bf{s}}_{1,2}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^3}}}d{\bf{n}}d{\bf{n}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}d{\bf{n}}d{\bf{n}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}d{\bf{n}}d{\bf{s}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}d{\bf{s}}d{\bf{n}}d{\bf{n}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}d{\bf{n}}d{\bf{s}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}d{\bf{s}}d{\bf{n}}d{\bf{s}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}d{\bf{s}}d{\bf{s}}d{\bf{n}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}d{{\bf{n}}_1}{d^2}{{\bf{s}}_{2,3}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}d{{\bf{n}}_2}{d^2}{{\bf{s}}_{1,3}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}d{{\bf{n}}_3}{d^2}{{\bf{s}}_{1,2}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}d{\bf{s}}d{\bf{s}}d{\bf{s}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}{d^2}{{\bf{s}}_{2,3}}d{{\bf{s}}_1} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}{d^2}{{\bf{s}}_{1,3}}d{{\bf{s}}_2} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}{d^2}{{\bf{s}}_{1,2}}d{{\bf{s}}_3}$

## Consequently,
along the homogenious equilibrium manifold ...

### (3-1) ${\left. {\frac{{{\partial ^3}G}}{{\partial {T^3}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {T^3}}} + 3\frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + 3\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}} + 3\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}} + 3\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}}\frac{{d{\bf{s}}}}{{dT}}$   
Code generates a scalar.

In [ ]:
d3gdt2ds = G.diff(T,2).diff(s)
result  = G.diff(T,3)
result += 3*(dsdt*d3gdt2ds)[0]
result += 3*(dsdt*d3gdtds2*(dsdt.T))[0]
term = []
for i in range(0,ns):
    term.append(dsdt*d3gds3[i]*(dsdt.T))
result += (dsdt*sym.Matrix(term))[0]
result += 3*(d2sdt2*d2gdtds)[0]
result += 3*(d2sdt2*d2gds2*(dsdt.T))[0]
result.simplify()

### (3-2) ${\left. {\frac{{{\partial ^3}G}}{{\partial {T^2}\partial P}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {T^2}\partial P}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}} + 2\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}}\frac{{d{\bf{s}}}}{{dP}}$

In [ ]:
d3gdtdpds = G.diff(T).diff(P).diff(s)
result  = G.diff(T,2).diff(P)
result +=   (dsdp*d3gdt2ds)[0]
result += 2*(dsdt*d3gdtdpds)[0]
result += 2*(dsdt*d3gdtds2*(dsdp.T))[0]
result +=   (dsdt*d3gdpds2*(dsdp.T))[0]
term = []
for i in range(0,ns):
    term.append(dsdt*d3gds3[i]*(dsdt.T))
result += (dsdp*sym.Matrix(term))[0]
result += 2*(d2sdtdp*d2gdtds)[0]
result +=   (d2sdt2*d2gdpds)[0]
result += 2*(d2sdtdp*d2gds2*(dsdt.T))[0]
result +=   (d2sdt2*d2gds2*(dsdp.T))[0]
result.simplify()

### (3-3) ${\left. {\frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{n}}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 2\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{T^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result  = G.diff(T,2).diff(n)
result +=   dsdn*d3gdt2ds
result += 2*(dsdt*d3gdtdnds).T
result += 2*(dsdt*d3gdtds2*(dsdn.T)).T
term = []
for i in range(0,nc):
    term.append(dsdt*d3gdnds2[i]*(dsdt.T))
result += sym.Matrix(term)
term = []
for i in range(0,ns):
    term.append(dsdt*d3gds3[i]*(dsdt.T))
result += dsdn*sym.Matrix(term)
result += 2*d2sdtdn*d2gdtds
result +=   (d2sdt2*d2gdnds).T
result += 2*d2sdtdn*d2gds2*(dsdt.T)
result +=   (d2sdt2*d2gds2*(dsdn.T)).T
result.simplify()

### (3-4) ${\left. {\frac{{{\partial ^3}G}}{{\partial T\partial {P^2}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial T\partial {P^2}}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}} + 2\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}}\frac{{d{\bf{s}}}}{{dP}}$

In [ ]:
d3gdp2ds = G.diff(P,2).diff(s)
result  = G.diff(T).diff(P, 2)
result +=   (dsdp*d3gdtdpds)[0]
result +=   (dsdt*d3gdtdpds)[0]
result +=   (dsdp*d3gdtds2*(dsdp.T))[0]
result +=   (dsdt*d3gdp2ds)[0]
result += 2*(dsdt*d3gdpds2*(dsdp.T))[0]
term = []
for i in range(0,ns):
    term.append(dsdp*d3gds3[i]*(dsdt.T))
result += (dsdp*sym.Matrix(term))[0]
result +=   (d2sdp2*d2gdtds)[0]
result += 2*(d2sdtdp*d2gdpds)[0]
result +=   (d2sdp2*d2gds2*(dsdt.T))[0]
result += 2*(d2sdtdp*d2gds2*(dsdp.T))[0]
result.simplify()

### (3-5) ${\left. {\frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{n}}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTdP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result  = G.diff(T).diff(P).diff(n)
result +=  dsdn*d3gdtdpds
result += (dsdp*d3gdtdnds).T
result +=  dsdn*d3gdtds2*(dsdp.T)
result += (dsdt*d3gdpdnds).T
result +=  dsdn*d3gdpds2*(dsdt.T)
term = []
for i in range(0,nc):
    term.append(dsdt*d3gdnds2[i]*(dsdp.T))
result += sym.Matrix(term)
term = []
for i in range(0,ns):
    term.append(dsdt*d3gds3[i]*(dsdp.T))
result +=  dsdn*sym.Matrix(term)
result +=  d2sdpdn*d2gdtds
result +=  d2sdtdn*d2gdpds
result += (d2sdtdp*d2gdnds).T
result += d2sdpdn*d2gds2*(dsdt.T)
result += d2sdtdn*d2gds2*(dsdp.T)
result += (d2sdtdp*d2gds2*(dsdn.T)).T
result

### (3-6) ${\left. {\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{n}}^2}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{n}}^2}}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 2\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{n^2}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{n^2}}}\frac{{d{\bf{s}}}}{{dT}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTd{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result  = sym.Matrix([[G.diff(T).diff(x).diff(y) for x in n] for y in n])
result += 2*dsdn*d3gdtdnds
result +=   dsdn*d3gdtds2*(dsdn.T)
for i in range(0,ns):
    result += sym.Matrix([[G.diff(x).diff(y).diff(s[i])*dsdt[i] for x in n] for y in n])
term = []
for i in range(0,nc):
    term.append(dsdt*d3gdnds2[i]*(dsdn.T))
result += 2*sym.Matrix(term)
for i in range(0,ns):
    result += dsdn*d3gds3[i]*(dsdn.T)*dsdt[i]
result += 2*d2sdtdn*d2gdnds
for i in range(0,ns):
    result += d2sdn2[i]*d2gdtds[i]
for i in range(0,ns):
    result += ((dsdt*d2gds2).row(i))[0]*d2sdn2[i]
result += 2*d2sdtdn*d2gds2*(dsdn.T)
result

### (3-7) ${\left. {\frac{{{\partial ^3}G}}{{\partial {P^3}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {P^3}}} + 3\frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + 3\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + 3\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}} + 3\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}}\frac{{d{\bf{s}}}}{{dP}}$

In [ ]:
result  = G.diff(P,3)
result += 3*(dsdp*d3gdp2ds)[0]
result += 3*(dsdp*d3gdpds2*(dsdp.T))[0]
term = []
for i in range(0,ns):
    term.append(dsdp*d3gds3[i]*(dsdp.T))
result += (dsdp*sym.Matrix(term))[0]
result += 3*(d2sdp2*d2gdpds)[0]
result += 3*(d2sdp2*d2gds2*(dsdp.T))[0]
result.simplify()

### (3-8) ${\left. {\frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{n}}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 2\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{P^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result  = G.diff(P,2).diff(n)
result +=   dsdn*d3gdp2ds
result += 2*(dsdp*d3gdpdnds).T
term = []
for i in range(0,nc):
    term.append(dsdp*d3gdnds2[i]*(dsdp.T))
result += sym.Matrix(term)
result += 2*(dsdp*d3gdpds2*(dsdn.T)).T
term = []
for i in range(0,ns):
    term.append(dsdp*d3gds3[i]*(dsdp.T))
result += dsdn*sym.Matrix(term)
result += 2*d2sdpdn*d2gdpds
result +=   (d2sdp2*d2gdnds).T
result += 2*d2sdpdn*d2gds2*(dsdp.T)
result +=   (d2sdp2*d2gds2*(dsdn.T)).T
result.simplify()

### (3-9) ${\left. {\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{n}}^2}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{n}}^2}}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {\bf{n}}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{dP}} + 2\frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dP}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}}{\frac{{d{\bf{s}}}}{{dP}}_1} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dPd{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result  = sym.Matrix([[G.diff(P).diff(x).diff(y) for x in n] for y in n])
result += 2*dsdn*d3gdpdnds
result +=   dsdn*d3gdpds2*(dsdn.T)
for i in range(0,ns):
    result += sym.Matrix([[G.diff(x).diff(y).diff(s[i])*dsdp[i] for x in n] for y in n])
term = []
for i in range(0,nc):
    term.append(dsdp*d3gdnds2[i]*(dsdn.T))
result += 2*sym.Matrix(term)
for i in range(0,ns):
    result += dsdn*d3gds3[i]*(dsdn.T)*dsdp[i]
result += 2*d2sdpdn*d2gdnds
for i in range(0,ns):
    result += d2sdn2[i]*d2gdpds[i]
for i in range(0,ns):
    result += ((dsdp*d2gds2).row(i))[0]*d2sdn2[i]
result += 2*d2sdpdn*d2gds2*(dsdn.T)
result

### (3-10) ${\left. {\frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^3}}}} \right|_{eq}} = \frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^3}}} + 3\frac{{{\partial ^3}G}}{{\partial {{\bf{n}}^2}\partial {\bf{s}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 3\frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + 3\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}} + 3\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{{\bf{n}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}$

In [ ]:
result = []
for i in range(0,nc):
    entry  = sym.Matrix(list([[G.diff(n[i]).diff(x).diff(y) for x in n] for y in n]))
    for j in range(0,ns):
        entry += 3*sym.Matrix([[G.diff(x).diff(y).diff(s[j]) for x in n] for y in n])*((dsdn.row(i))[j])
    entry += 3*dsdn*d3gdnds2[i]*(dsdn.T)
    for j in range(0,ns):
        entry += dsdn*d3gds3[j]*(dsdn.T)*((dsdn.row(i))[j])
    for j in range(0,ns):
        entry += 3*d2sdn2[j]*((d2gdnds.col(i))[j])
    for j in range(0,ns):
        entry += 3*((dsdn.row(i))*d2gds2)[j]*d2sdn2[j]
    result.append(entry)
result

# Generic formulation using Faà di Bruno's formula
generalizing the chain rule to higher derivatives ([reference](https://en.wikipedia.org/wiki/Faà_di_Bruno%27s_formula))

### $G\left( {T,P,{n_1}, \cdots ,{n_{nc}},{s_1}, \cdots ,{s_{ns}}} \right)$

### $dG = \frac{{\partial G}}{{\partial T}}dT + \frac{{\partial G}}{{\partial P}}dP + \sum\limits_{j = 1}^{nc} {\frac{{\partial G}}{{\partial {n_j}}}d{n_j}}  + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}d{s_i}}$

## First derivatives:
### $\frac{{\partial G}}{{\partial x}} = {\left. {\frac{{\partial G}}{{\partial x}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial x}}}$
where $x$ can be one of $T$, $P$, or $n_j$, and the partual derivatives hold all other variables constant except $x$.  For the first term on the right-hand-side of the expression, the derivative is taken holding all $s_i$ constant.

## Second derivatives:
Differentiating the two terms in $\frac{{\partial G}}{{\partial {x_j}}} = {\left. {\frac{{\partial G}}{{\partial {x_j}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}}$ gives:  
- $d\left( {{{\left. {\frac{{\partial G}}{{\partial {x_j}}}} \right|}_{{s_i}}}} \right) \Rightarrow {\left. {\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {x_k}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^2}G}}{{\partial {s_i}\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}} \right)}$ and  
- $d\left( {\frac{{\partial G}}{{\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}} \right) \Rightarrow \frac{{{\partial ^2}G}}{{\partial {s_i}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^2}G}}{{\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{\partial G}}{{\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}$  

Together, they provide an expression for the second derivative:  
### $\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {x_k}}} = {\left. {\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {x_k}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}G}}{{\partial {x_k}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}}  + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}}$

## Third derivatives:
### $\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^3}G}}{{\partial {x_k}\partial {x_l}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_l}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \right)} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_k}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_l}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial s_i^3}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}} + \frac{{{\partial ^2}G}}{{\partial {x_k}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}} + \frac{{{\partial ^2}G}}{{\partial {x_l}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_j}\partial {x_k}\partial {x_l}}}}$

## Fourth derivatives:
### $\frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}\partial {x_m}}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}\partial {x_m}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^4}G}}{{\partial {x_k}\partial {x_l}\partial {x_m}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_l}\partial {x_m}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial {x_m}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial {x_m}}}} \right)} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_l}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_m}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^4}G}}{{\partial {x_k}\partial {x_l}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_k}\partial {x_m}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^4}G}}{{\partial {x_l}\partial {x_m}\partial s_i^2}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}} \right)} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^4}G}}{{\partial {x_j}\partial s_i^3}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_k}\partial s_i^3}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_l}\partial s_i^3}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^4}G}}{{\partial {x_m}\partial s_i^3}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^4}G}}{{\partial s_i^4}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \frac{{\partial {s_i}}}{{\partial {x_m}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_m}}} + \frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_l}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_m}}} + \frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_m}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_k}\partial {x_l}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_m}}} + \frac{{{\partial ^3}G}}{{\partial {x_k}\partial {x_m}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_l}\partial {x_m}\partial {s_i}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_m}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial {x_k}\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_m}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial {x_l}\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_m}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial {x_m}\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial s_i^3}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_m}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_m}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}} \right)} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^2}G}}{{\partial {x_j}\partial {s_i}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_k}\partial {x_l}\partial {x_m}}} + \frac{{{\partial ^2}G}}{{\partial {x_k}\partial {s_i}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_j}\partial {x_l}\partial {x_m}}} + \frac{{{\partial ^2}G}}{{\partial {x_l}\partial {s_i}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_j}\partial {x_k}\partial {x_m}}} + \frac{{{\partial ^2}G}}{{\partial {x_m}\partial {s_i}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_j}\partial {x_k}\partial {x_l}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial s_i^2}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_l}\partial {x_m}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_m}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_m}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{\partial G}}{{\partial {s_i}}}\frac{{{\partial ^4}{s_i}}}{{\partial {x_j}\partial {x_k}\partial {x_l}\partial {x_m}}}} $

## Order parameter first derivatives
$\frac{{\partial \frac{{\partial G}}{{\partial {s_u}}}}}{{\partial x}} = {\left. {\frac{{\partial \frac{{\partial G}}{{\partial {s_u}}}}}{{\partial x}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\frac{{\partial \frac{{\partial G}}{{\partial {s_u}}}}}{{\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial x}}}$  
which reduces to  
### $0 = \frac{{{\partial ^2}G}}{{\partial x\partial {s_u}}} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial {s_u}\partial {s_i}}}\frac{{\partial {s_i}}}{{\partial x}}} \forall {s_u}$

## Order parameter second derivatives
### $0 = {\left. {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {x_k}\partial {s_u}}}} \right|_{{s_i}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {s_i}\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^3}G}}{{\partial {x_k}\partial {s_i}\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial s_i^2\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial {s_i}\partial {s_u}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}}$

## Order parameter third derivatives
### $0 = \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial {x_l}\partial {s_u}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^4}G}}{{\partial {x_k}\partial {x_l}\partial {s_i}\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_l}\partial {s_i}\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^4}G}}{{\partial {x_j}\partial {x_k}\partial {s_i}\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \right)} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^4}G}}{{\partial {x_j}\partial s_i^2\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^4}G}}{{\partial {x_k}\partial s_i^2\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_l}}} + \frac{{{\partial ^4}G}}{{\partial {x_l}\partial s_i^2\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^4}G}}{{\partial s_i^3\partial {s_u}}}\frac{{\partial {s_i}}}{{\partial {x_j}}}\frac{{\partial {s_i}}}{{\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} + \sum\limits_{i = 1}^{ns} {\left( {\frac{{{\partial ^3}G}}{{\partial {x_j}\partial {s_i}\partial {s_u}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_k}\partial {s_i}\partial {s_u}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}} + \frac{{{\partial ^3}G}}{{\partial {x_l}\partial {s_i}\partial {s_u}}}\frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}} \right)} + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^3}G}}{{\partial s_i^2\partial {s_u}}}\left( {\frac{{{\partial ^2}{s_i}}}{{\partial {x_k}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_j}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_l}}}\frac{{\partial {s_i}}}{{\partial {x_k}}} + \frac{{{\partial ^2}{s_i}}}{{\partial {x_j}\partial {x_k}}}\frac{{\partial {s_i}}}{{\partial {x_l}}}} \right)}  + \sum\limits_{i = 1}^{ns} {\frac{{{\partial ^2}G}}{{\partial {s_i}\partial {s_u}}}\frac{{{\partial ^3}{s_i}}}{{\partial {x_j}\partial {x_k}\partial {x_l}}}}$

# Required Parameter Derivatives
By analogy, if *w* is some model parameter, then
### ${\bf{0}} = \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dw}}$

From which the required second order Gibbs free energy parameter derivatives are:  

### $\frac{{{\partial ^2}G}}{{\partial T\partial w}} = {\left. {\frac{{{\partial ^2}G}}{{\partial T\partial w}}} \right|_{{s_i}}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}}$
### $\frac{{{\partial ^2}G}}{{\partial P\partial w}} = {\left. {\frac{{{\partial ^2}G}}{{\partial P\partial w}}} \right|_{{s_i}}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}}$
### $\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial w}} = {\left. {\frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial w}}} \right|_{{s_i}}} + \frac{{{\partial ^2}G}}{{\partial {\bf{n}}\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial {\bf{n}}}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial {\bf{n}}}}\frac{{\partial {\bf{s}}}}{{\partial w}}$

The second order parameter derivatives are:  

### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial T\partial w\partial \bf{s}}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dw}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dT}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dT}}\frac{{d{\bf{s}}}}{{dw}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dTdw}}$
### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial w\partial P\partial \bf{s}}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dw}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{dw}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{dwdP}}$
### ${\bf{0}} = \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial w\partial \bf{s}}} + \frac{{{\partial ^3}G}}{{\partial {\bf{n}}\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{dP}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{d{\bf{s}}}}{{d{\bf{n}}}}\frac{{d{\bf{s}}}}{{dw}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{d^2}{\bf{s}}}}{{d{\bf{n}}dw}}$

From which the required third order Gibbs free energy derivatives are:  

$\frac{{{\partial ^3}G}}{{\partial {T^2}\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {T^2}\partial w}}} \right|_{{s_i}}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}}$  

$\frac{{{\partial ^3}G}}{{\partial T\partial P\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial T\partial P\partial w}}} \right|_{{s_i}}} + \frac{{{\partial ^3}G}}{{\partial P\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial T\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}}$  

$\frac{{{\partial ^3}G}}{{\partial {P^2}\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {P^2}\partial w}}} \right|_{{s_i}}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}} + 2\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}}$

The third order parameter derivatives are:  

${\bf{0}} = \frac{{{\partial ^4}G}}{{\partial {T^2}\partial w\partial {\bf{s}}}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {T^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}} + 2\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {T^2}\partial w}}$  

${\bf{0}} = \frac{{{\partial ^4}G}}{{\partial T\partial P\partial w\partial {\bf{s}}}} + \frac{{{\partial ^4}G}}{{\partial P\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial T\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial T\partial P\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial T\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial P\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial T\partial P\partial w}}$  

${\bf{0}} = \frac{{{\partial ^4}G}}{{\partial {P^2}\partial w\partial {\bf{s}}}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {P^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}} + 2\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {P^2}\partial w}}$

From which the required fourth order Gibbs free energy derivatives are:  

$\frac{{{\partial ^4}G}}{{\partial {T^3}\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {T^3}\partial w}}} \right|_{{s_i}}} + 3\frac{{{\partial ^4}G}}{{\partial {T^2}\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {T^3}\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^4}G}}{{\partial {T^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^4}G}}{{\partial T\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 3\frac{{{\partial ^4}G}}{{\partial T\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial T\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}} + 6\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 3\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 3\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 3\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {T^2}\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {T^3}}} + 3\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}$  

$\frac{{{\partial ^4}G}}{{\partial {T^2}\partial P\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {T^2}\partial P\partial w}}} \right|_{{s_i}}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial P\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {T^2}\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {T^2}\partial P\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial {T^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial P\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial P\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 2\frac{{{\partial ^4}G}}{{\partial T\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial P\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial {T^2}\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial P\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}} + \frac{{{\partial ^3}G}}{{\partial P\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 2\frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 2\frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial T\partial P\partial w}} + \frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {T^2}\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {T^2}\partial P}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {T^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial P}}$  

$\frac{{{\partial ^4}G}}{{\partial {P^2}\partial T\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {P^2}\partial T\partial w}}} \right|_{{s_i}}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial T\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {P^2}\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {P^2}\partial T\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial {P^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial T\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial T\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 2\frac{{{\partial ^4}G}}{{\partial P\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial T\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial T\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}} + \frac{{{\partial ^3}G}}{{\partial T\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 2\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial T\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}} + 2\frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial T}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 2\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial T}} + \frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 2\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial P\partial T\partial w}} + \frac{{{\partial ^2}G}}{{\partial T\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {P^2}\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {P^2}\partial T}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial T\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + \frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial T}}$  
  
$\frac{{{\partial ^4}G}}{{\partial {P^3}\partial w}} = {\left. {\frac{{{\partial ^3}G}}{{\partial {P^3}\partial w}}} \right|_{{s_i}}} + 3\frac{{{\partial ^4}G}}{{\partial {P^2}\partial w\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {P^3}\partial {\bf{s}}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^4}G}}{{\partial {P^2}\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^4}G}}{{\partial P\partial w\partial {{\bf{s}}^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 3\frac{{{\partial ^4}G}}{{\partial P\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + \frac{{{\partial ^4}G}}{{\partial w\partial {{\bf{s}}^3}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + \frac{{{\partial ^4}G}}{{\partial {{\bf{s}}^4}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial {P^2}\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial P\partial w\partial {\bf{s}}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}} + 6\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 3\frac{{{\partial ^3}G}}{{\partial P\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial w\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 3\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial w}} + 3\frac{{{\partial ^3}G}}{{\partial {{\bf{s}}^3}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}\frac{{\partial {\bf{s}}}}{{\partial P}}\frac{{\partial {\bf{s}}}}{{\partial P}} + 3\frac{{{\partial ^2}G}}{{\partial P\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {P^2}\partial w}} + \frac{{{\partial ^2}G}}{{\partial w\partial {\bf{s}}}}\frac{{{\partial ^3}{\bf{s}}}}{{\partial {P^3}}} + 3\frac{{{\partial ^2}G}}{{\partial {{\bf{s}}^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial {P^2}}}\frac{{{\partial ^2}{\bf{s}}}}{{\partial P\partial w}}$